## 가위, 바위, 보 인식하기!

#### Load Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow, Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image = tf.keras.utils

# Settings
src_dir = "./dataset/"
%matplotlib inline
!nvidia-smi

#### Load Images

In [ ]:
generator = ImageDataGenerator(validation_split=0.2, rescale=1/255, shear_range=0.2, zoom_range=0.2, rotation_range=20, fill_mode='nearest')
train_data = generator.flow_from_directory(src_dir, batch_size=32, target_size=(150,150), subset='training')
val_data = generator.flow_from_directory(src_dir, batch_size=32,  target_size=(150,150), subset='validation')

#### Create Model

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(3, activation="softmax")
])

#### Early Stop Callback

In [ ]:
class Callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.90:
            print("\nReached %2.2f%% accuracy, training has been stop" %(logs.get('accuracy')*100))
            self.model.stop_training = True
callbacks = Callbacks()

#### Model Compile

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.optimizers.Adam(),
    metrics=['accuracy']
)

#### Run Learning

In [ ]:
historyModel = model.fit(
    train_data,
    steps_per_epoch = 25,
    epochs = 20,
    validation_data = val_data,
    validation_steps = 5,
    verbose = 1,
    callbacks = [callbacks]
)

#### Predict

In [ ]:
img = image.load_img("paper.jpg", target_size = (150, 150))

imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = x / 255.0
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images) 
print(classes)
n = np.argmax(classes)
match n:
    case 0:
        print("Paper")
    case 0:
        print("Rock")
    case 0:
        print("Scissors")